# Análise de sentimento usando word embeddings - IMDB

Anteriormente vimos uma primeira solução de análise de sentimento utilizando *bag of words*.
Agora iremos ilustrar o uso de *word embeddings* como vetor de atributos latentes de cada palavra.

Duas soluções são propostas neste exercícios:

1. Utilizando rede neural com camadas densas
2. Utilizando camadas convolucionais 1D

Diferentemente da solução apresentada com *bag of words*, nestas duas soluções, é necessário que o
número de palavras seja o mesmo para cada amostra. Para isso, limita-se o número de palavras e caso
o número de palavras for menor, completa-se com um código especial e palavras além do limite são
descartadas.

## Importação dos pacotes

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os, sys
import json
import numpy as np
import pandas as pd
import numpy.random as nr

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR, StepLR
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable

from torchvision import datasets, transforms, models

import lib.pytorch_trainer as ptt

use_gpu = torch.cuda.is_available()
print('GPU available:', use_gpu)

GPU available: False


## Dataset IMDB

### Lendo do disco

O dataset é composto de 25 mil amostras de treinamento e 25 mil amostras de teste.
Cada amostra possui um texto de tamanho que varia entre 11 e 2494 palavras. 
Cada amostra tem um rótulo igual a 1 para denominar sentimento positivo e 0 para sentimento negativo.

In [2]:
word_index = json.load(open('/data/datasets/IMDB/imdb_word_index.json'))
data = np.load('/data/datasets/IMDB/imdb.npz')
x_test, x_train, y_train, y_test = data['x_test'], data['x_train'], data['y_train'], data['y_test']

n_words = len(word_index)
n_train = x_train.shape[0]
n_test  = x_test.shape[0]

word_list = [None for i in range(n_words+1)]
for k, v in word_index.items():
    word_list[v] = k

n_words, n_train, n_test

(88584, 25000, 25000)

In [3]:
def print_stats(x_train, x_test, word_list=None):
    print('Train word index limits:', min([min(s) for s in x_train]), max([max(s) for s in x_train]))
    print('Test word index limits:', min([min(s) for s in x_test]), max([max(s) for s in x_test]))
    print('\nTrain sequence length limits:', min([len(x) for x in x_train]), max([len(x) for x in x_train]))
    print('Test sequence length limits:', min([len(x) for x in x_test]), max([len(x) for x in x_test]))
    if word_list:
        print('\nMost frequent words:', word_list[1:11])
    
print_stats(x_train, x_test, word_list)

Train word index limits: 1 88584
Test word index limits: 1 88581

Train sequence length limits: 10 2493
Test sequence length limits: 6 2314

Most frequent words: ['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']


### Limitando o vocabulário

Retiramos das sequências as palavras com índice maior que o valor especificado em `voc_size`.

In [4]:
voc_size = 5000

xtra = [[w for w in x if (w < voc_size)] for x in x_train]
xval = [[w for w in x if (w < voc_size)] for x in x_test]
print_stats(xtra, xval)

Train word index limits: 1 4999
Test word index limits: 1 4999

Train sequence length limits: 9 1973
Test sequence length limits: 6 2113


### Obtendo sequências de mesmo comprimento

Fazemos com que todas as sequências tenham o mesmo comprimento, especificado em `seq_len`. As sequências mais longas que `seq_len` são truncadas e as menores, completadas com zeros.

In [5]:
def pad_sequences(sequences, seq_len, post_pad=True, fill_value=0):
    new_seq = []
    for seq in sequences:
        n = len(seq)
        if n > seq_len:
            if post_pad:
                new_seq.append(seq[-seq_len:])
            else:
                new_seq.append(seq[:seq_len])
        else:
            zseq = [fill_value for i in range(seq_len)]
            if post_pad:
                zseq[-n:] = seq
            else:
                zseq[:n] = seq
            new_seq.append(zseq)
    return new_seq
    

In [6]:
seq_len = 500
xtra = pad_sequences(xtra, seq_len, post_pad=True)
xval = pad_sequences(xval, seq_len, post_pad=True)
print_stats(xtra, xval)

Train word index limits: 0 4999
Test word index limits: 0 4999

Train sequence length limits: 500 500
Test sequence length limits: 500 500


### Convertendo para tensores

In [7]:
Xtrain = torch.from_numpy(np.array(xtra, np.int))
Xvalid = torch.from_numpy(np.array(xval, np.int))
ytrain = torch.from_numpy(np.array(y_train, np.int))
yvalid = torch.from_numpy(np.array(y_test, np.int))

Xtrain.size(), Xtrain.max(), Xvalid.size(), Xvalid.max()

(torch.Size([25000, 500]), 4999, torch.Size([25000, 500]), 4999)

## Rede Densa


In [19]:
class MySimpleNet(nn.Module):

    def __init__(self, seq_len=seq_len, voc_size=voc_size, embed_dim=None):
        super().__init__()
        self.flat_size = seq_len * embedding_dim        
        self.emb = nn.Embedding(voc_size, embed_dim)
        nn.init.xavier_uniform(self.emb.weight)
        self.fc1 = nn.Linear(self.flat_size, 128)
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.emb(x)
        x = x.view(-1, self.flat_size)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        x = self.fc2(x)
        return x


In [32]:
class ModelRNN(nn.Module):
    
    def __init__(self, hidden_size, voc_size=voc_size, embed_dim = None):
        super().__init__()
        self.emb = nn.Embedding(voc_size, embed_dim)
        nn.init.xavier_uniform(self.emb.weight)
        self.rnn = nn.RNN(input_size=embed_dim,hidden_size=hidden_size, dropout=0.05,batch_first=True)
        self.fc1 = nn.Linear(hidden_size,2)
        
    def forward(self, x):
        x = self.emb(x)
        print('x.:',x.data.shape)
        _,x = self.rnn(x)
        x = self.fc1(x)
        x = torch.squeeze(x,0)
        return x
    
model_rnn = ModelRNN(100,5000,50)
model_rnn

ModelRNN (
  (emb): Embedding(5000, 50)
  (rnn): RNN(50, 100, batch_first=True, dropout=0.05)
  (fc1): Linear (100 -> 2)
)

## Testando predict com uma amostra

In [33]:
print(Xtrain[0:2].size())
y = model_rnn(Variable(Xtrain[0:2]))
y

torch.Size([2, 500])
x.: torch.Size([2, 500, 50])


Variable containing:
 0.0164 -0.1700
 0.0185 -0.1760
[torch.FloatTensor of size 2x2]

### Treinamento

In [35]:
trainIt = True
resetIt = True

embedding_dim = 50
batch_size = 100
n_epochs = 10

# Callbacks
# ---------
state_fn = '../../models/sentimento_rnn'
accuracy_cb = ptt.AccuracyMetric()
chkpt_cb = ptt.ModelCheckpoint(state_fn, reset=resetIt, verbose=1)
print_cb = ptt.PrintCallback()
plot_cb = ptt.PlotCallback()

# Model, optimizer and learning rate scheduler
# --------------------------------------------
optimizer = torch.optim.Adam(model_rnn.parameters(), lr=1e-4, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

# Network trainer
# ---------------
training_parameters = {
    'model':         model_rnn, 
    'criterion':     nn.CrossEntropyLoss(),
    'optimizer':     optimizer, 
    'lr_scheduler':  scheduler, 
    'callbacks':     [accuracy_cb, chkpt_cb, print_cb],
}
trainer = ptt.DeepNetTrainer(**training_parameters)


In [ ]:
if trainIt:
    trainer.fit(n_epochs, Xtrain, ytrain, valid_data=(Xvalid, yvalid), batch_size=batch_size)
else:
    print('\nTraining disabled.\nThis model was trained for {} epochs.'.format(trainer.last_epoch))

Start training for 10 epochs
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])
x.: torch.Size([100, 500, 50])


### Avaliação

In [11]:
if 'ModelCheckpoint' in [cb.__class__.__name__ for cb in trainer.callbacks]:
    trainer.load_state(state_fn)

rmetrics = trainer.evaluate(Xtrain, ytrain, metrics=[accuracy_cb])
print('Model training set accuracy after training: {:.5f}'.format(rmetrics['acc']))
print()
rmetrics = trainer.evaluate(Xvalid, yvalid, metrics=[accuracy_cb])
print('Model validation set accuracy after training: {:.5f}'.format(rmetrics['acc']))

evaluate: 2499/2499 ok
Model training set accuracy after training: 0.95360

evaluate: 2499/2499 ok
Model validation set accuracy after training: 0.88388


## Rede convolucional

In [44]:
class MyNet(nn.Module):
    
    def __init__(self, seq_len=seq_len, voc_size=voc_size, embed_dim=embedding_dim, 
                 n_conv_filters=128, conv_kernel_size=5):
        super().__init__()
        
        k = conv_kernel_size - 1
        n = (((seq_len - k) // 2 - k) // 2 - k) // 2
        self.flat_size = n * n_conv_filters
        
        self.embedding = nn.Embedding(voc_size, embed_dim)
        nn.init.xavier_uniform(self.embedding.weight)

        self.conv_net = nn.Sequential(
            nn.Conv1d(embed_dim, n_conv_filters, conv_kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.5),
            
            nn.Conv1d(n_conv_filters, n_conv_filters, conv_kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.5),
            
            nn.Conv1d(n_conv_filters, n_conv_filters, conv_kernel_size),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.5),
        )
        
        self.fc_net = nn.Sequential(
            nn.Linear(self.flat_size, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 2),
        )

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(1, 2)
        x = self.conv_net(x)
        x = x.view(-1, self.flat_size)
        x = self.fc_net(x)
        return x

### Treinamento

In [45]:
trainIt = True
resetIt = True

# embedding_dim = 50
# batch_size = 100
# n_epochs = 15

# Callbacks
# ---------
state_fn = '../../models/sentimento_3'
accuracy_cb = ptt.AccuracyMetric()
chkpt_cb = ptt.ModelCheckpoint(state_fn, reset=resetIt, verbose=1)
print_cb = ptt.PrintCallback()
plot_cb = ptt.PlotCallback()

# Model, optimizer and learning rate scheduler
# --------------------------------------------
model = MyNet(seq_len, voc_size, embedding_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=0)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

# Network trainer
# ---------------
training_parameters = {
    'model':         model, 
    'criterion':     nn.CrossEntropyLoss(),
    'optimizer':     optimizer, 
    'lr_scheduler':  scheduler, 
    'callbacks':     [accuracy_cb, chkpt_cb, print_cb],
}
trainer = ptt.DeepNetTrainer(**training_parameters)


In [46]:
if trainIt:
    trainer.fit(n_epochs, Xtrain, ytrain, valid_data=(Xvalid, yvalid), batch_size=batch_size)
else:
    print('\nTraining disabled.\nThis model was trained for {} epochs.'.format(trainer.last_epoch))

Start training for 15 epochs
  1:  20.1s   T: 0.54530 0.67476   V: 0.29361 0.87632 best
  2:  20.0s   T: 0.26240 0.89720   V: 0.25804 0.89400 best
  3:  20.0s   T: 0.20004 0.92428   V: 0.26077 0.89016 
  4:  20.0s   T: 0.15627 0.94348   V: 0.31013 0.88424 
  5:  20.0s   T: 0.11032 0.95872   V: 0.31990 0.88600 
  6:  20.0s   T: 0.08112 0.97168   V: 0.35944 0.87540 
  7:  20.0s   T: 0.06145 0.97928   V: 0.41989 0.87900 
  8:  20.0s   T: 0.04737 0.98420   V: 0.49785 0.87260 
  9:  20.0s   T: 0.03775 0.98700   V: 0.56124 0.87476 
 10:  20.0s   T: 0.02887 0.99004   V: 0.62317 0.87572 
 11:  20.1s   T: 0.02146 0.99264   V: 0.64935 0.87852 
 12:  20.0s   T: 0.01807 0.99312   V: 0.76701 0.87420 
 13:  20.1s   T: 0.01802 0.99420   V: 0.76433 0.87756 
 14:  20.1s   T: 0.01260 0.99508   V: 0.85571 0.87700 
 15:  20.1s   T: 0.01277 0.99528   V: 0.83723 0.87348 
Best model was saved at epoch 2 with loss 0.25804: ../../models/sentimento_3
Stop training at epoch: 15/15


### Avaliação

In [47]:
if 'ModelCheckpoint' in [cb.__class__.__name__ for cb in trainer.callbacks]:
    trainer.load_state(state_fn)

rmetrics = trainer.evaluate(Xtrain, ytrain, metrics=[ptt.AccuracyMetric()])
print('Model training set accuracy after training: {:.5f}'.format(rmetrics['acc']))
print()
rmetrics = trainer.evaluate(Xvalid, yvalid, metrics=[ptt.AccuracyMetric()])
print('Model validation set accuracy after training: {:.5f}'.format(rmetrics['acc']))

evaluate: 2499/2499 ok
Model training set accuracy after training: 0.94196

evaluate: 2499/2499 ok
Model validation set accuracy after training: 0.89400


## Resumo dos resultados

1. Experimento *bag of words*: 87% de acurácia
2. Experimento *word embeddings*, rede densa: 88%
3. Experimento *word embeddings*, rede convolucional: 89%